In [ ]:
import kagglehub
import pandas as pd
import matplotlib.pyplot as plt

# Download latest version
path = kagglehub.dataset_download("ethon0426/lending-club-20072020q1")

print("Path to dataset files:", path)

p = f"{path}/Loan_status_2007-2020Q3.gzip"


pd.set_option('display.max_columns', None)

df_borrowers = pd.read_csv(p)
print(len(df_borrowers))
df_borrowers.head()

In [ ]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
p = f"{path}/LCDataDictionary.xlsx"

df_descriptions = pd.read_excel(p)
df_descriptions

As we don't have the roll rates for loans to see the best past due to determine if a loan is good or bad, from description ´loan_status´ could be a good variable to use as targe. Let's see it's values.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
# Count values and plot as barplot
df_borrowers['loan_status'].value_counts().plot(kind='bar', color='skyblue')

plt.xlabel('Loan Status')
plt.ylabel('Count')
plt.title('Distribution of Loan Status in 12-Month Dataset')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

This variable can be taken as the target, using the labels related to "charged off" to flag a loan as bad and the "fully paid" and "current" labels to mark a loan a not defaulter. To avoid noise, some labels  like "In Grace Period", "Late (16-30 days)" and "Late (31-120 days)" can be used to flag the loan as indetermined.

Also we are going to take the loan with at least 12 months of lifetime (the common sandard on personal loans) to select the first group of loans to be modeled. As the databse doesn't have 'cuttof date' we are going to use as proxy of life of a credit ´issue_d´ and ´last_pymnt_d´ or ´last_credit_pull_d'

Next, loans are going to be selected and labeled as defaulted or not based on previous criteria.

In [ ]:
import pandas as pd

df = df_borrowers.copy()
for c in ['issue_d','last_pymnt_d','last_credit_pull_d']:
    df[c] = pd.to_datetime(df[c], errors='coerce')

# as-of for loan
as_of_row = df[['last_pymnt_d', 'last_credit_pull_d']].max(axis=1)
has_12m = (df['issue_d'] + pd.DateOffset(months=12)) <= as_of_row

default = {
    'Charged Off', 'Default',
    'Does not meet the credit policy: Status:Charged Off',  # <- opcional
}
non_default = {
    'Fully Paid', 'Current',
    'Does not meet the credit policy: Status:Fully Paid',   # <- opcional
}
indetermined = {'In Grace Period', 'Late (16-30 days)', 'Late (31-120 days)'}
policy_prefix = 'Does not meet the credit policy'

def label_row(s, has12):
    if pd.isna(s):
        return 'undetermined'
    if s in default:
        return 'default'
    if s in non_default and has12:
        return 'non_default'
    if (s in indetermined) or (s.startswith(policy_prefix) and s not in default and s not in non_default):
        return 'undetermined'

    return 'undetermined'

df['target'] = [label_row(s, h) for s, h in zip(df['loan_status'].fillna(''), has_12m)]

selected_loans_df = df[df['target'].isin(['default','non_default'])].copy()

In [ ]:
len(selected_loans_df)

In [ ]:
selected_loans_df['target'].value_counts().plot(kind='bar', color='skyblue')

plt.xlabel('Loan Status')
plt.ylabel('Count')
plt.title('Distribution of Loan Status in 12-Month Dataset')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
max(selected_loans_df.issue_d.unique())
min(selected_loans_df.issue_d.unique())

In [ ]:
import pandas as pd

df = selected_loans_df.copy()

# 1) Parsear fechas: 'Dec-2011' -> 2011-12-01
def parse_month(s):
    # intenta %b-%Y (Dec-2011); si no, deja que pandas infiera
    out = pd.to_datetime(s, format='%b-%Y', errors='coerce')
    if pd.isna(out):
        out = pd.to_datetime(s, errors='coerce')
    return out

df['issue_d'] = df['issue_d'].astype(str).map(parse_month)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
# Extract year
df['year'] = df['issue_d'].dt.year

# Count loans per year and target
loans_by_year_target = (
    df.groupby(['year', 'target'])
      .size()
      .unstack(fill_value=0)
      .sort_index()
)

# Normalize to percentage (100%)
loans_by_year_target_pct = loans_by_year_target.div(loans_by_year_target.sum(axis=1), axis=0) * 100

# Plot stacked bar (100%)
loans_by_year_target_pct.plot(kind='bar', stacked=True, figsize=(8, 5), colormap='tab20c')

# Labels and title
plt.xlabel('Year')
plt.ylabel('Percentage of Loans (%)')
plt.title('Loan Composition by Year and Target (100% Stacked)')
plt.xticks(rotation=45)
plt.legend(title='Target', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()


For train-test-oot spliting lets use the info of issued loans from 2015 and 2019.

In [ ]:
from pathlib import Path
# Create output folders
out = Path("data/processed"); out.mkdir(parents=True, exist_ok=True)

# Save as Parquet with fastparquet (preserves dtypes nicely)
train.to_parquet(out / "train.parquet", engine="fastparquet", index=False)
valid.to_parquet(out / "valid.parquet", engine="fastparquet", index=False)
test.to_parquet( out / "test.parquet",  engine="fastparquet", index=False)

print("Saved to:", list(out.glob("*.parquet")))